In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from datetime import timedelta
import warnings
from pathlib import Path
import os
import numpy as np
import pandas as pd
def fmt_float(x) :
    float_fmt_str = "{:6.2f}"
    return (float_fmt_str.format(x))

pd.set_option('float_format', fmt_float)
idx = pd.IndexSlice


`Load profile`  : we have load profiles of years 2015 till spring 2022 , However our assumption year is 2030 so we need to run regression model in                    order to get the predicted load profiles of year 2030 to give us more accurate insights.                    load profile. 


In [ ]:
pip install openpyxl

In [ ]:
#Used only Ireland demand while the SWIS 100 aggregated northern ireland with Ireland republic data
df_load_concated=pd.DataFrame()
cols = ['DateTime', 'GMT Offset', 'IE Demand']
for filename in os.listdir('../input/eirgrid-data/EirGrid'):
    df_load_concated=pd.concat([df_load_concated, pd.read_excel('../input/eirgrid-data/EirGrid/'+filename, usecols = cols)], axis=0)
df_load_concated=df_load_concated.rename(columns={'IE Demand':'IE', 'NI Demand':'NI'}) 

In [ ]:
#clean the dataframe 
df_load_concated=df_load_concated.dropna()

In [ ]:
def tz_fix(row):
  try:
    naive_timestamp = row['DateTime']
    gmt_offset = row['GMT Offset'] 
    utc_timestamp = naive_timestamp - timedelta(hours=float(gmt_offset))
        # float() conversion required for timedelta() argument!
        # Must SUBTRACT the GMT Offset to get GMT/UTC
    row['DateTime'] = utc_timestamp.tz_localize('UTC')
  except Exception as inst:
    print(F"Exception:\n {row}")
    print(inst)
  return row

In [ ]:
df_load_concated = df_load_concated.apply(tz_fix, axis=1).drop(columns='GMT Offset')
df_load_concated.set_index('DateTime', verify_integrity=True, inplace=True)

In [ ]:
#use electrical load for Ireland
elec_load = df_load_concated['IE'] 
elec_load_max = elec_load.max()
elec_load_mean = elec_load.mean()
elec_load_min = elec_load.min()
print('Mean electrical load is {}'.format(int(elec_load_mean))
      +' MW , while the maximum electrical load is {}'.format(int(elec_load_max))+' MW')

## Facebook prophet for prediction :

In [ ]:
#Copy df for prediction
df_for_prediction=df_load_concated.copy()

In [ ]:
#rename axis to match facebook prophet column names
df_for_prediction=df_for_prediction.rename_axis('ds').reset_index()

In [ ]:
df_for_prediction.columns=['ds','y']

In [ ]:
df_for_prediction=df_for_prediction.sort_values(by='ds')

In [ ]:
#remove the GMT shift from the datetime .
df_for_prediction['ds']=df_for_prediction['ds'].dt.tz_localize(None)
m = Prophet()
#Fit facebook prophet to the dataset. 
m.fit(df_for_prediction)
# #Make future predictions for 9 years. 
future = m.make_future_dataframe(periods=365*9*24,freq='H')
future.tail()
##Run inferences 
forecast = m.predict(future)
##Yhat is the predicted value while yhat lower and upper are 
##predictions upper and lower limits. 
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
forecast=forecast.sort_values(by='ds')
forecast=forecast.set_index('ds')
forecast.to_csv('./forecast_2030.csv')

### Conclusions: 
 
<br>
Our optimization algorithm will be built based on the predicted values for 2030.
<br>

# Validate Facebook prophet output

In [ ]:

df_forecast=pd.read_csv('../input/irforecast/forecast_2030.csv',parse_dates=True, index_col='ds')
df_forecast=df_forecast.resample("H").mean()
TWH_Summary=pd.DataFrame()
TWH_Summary['year']=""
TWH_Summary['value']=""
i=2022
while i<=2030:
    Year=i
    year_start='01-01-{}'.format(Year)
    year_end='31-12-{}'.format(Year)
    Truncated_df=df_forecast.truncate(before=year_start , after=year_end)
    Truncated_df=Truncated_df[['yhat']]
    TWH=Truncated_df['yhat'].sum()/(1e6)
    dummy_df=pd.DataFrame({"year":[Year],"value":[TWH]})
    TWH_Summary=TWH_Summary.append(dummy_df)
    i+=1
import plotly.express as px

fig = px.bar(TWH_Summary, x='year', y='value',
             title='FB Perophet Forecasted TWH ',labels={'value':'TOTAL TWH '},height=800)
fig.show()    